In [1]:
import mido
from abc import ABC, abstractmethod

In [2]:
class Note:
    NOTES = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
    OCTAVES = list(range(11))
    NOTES_IN_OCTAVE = len(NOTES)

    def __init__(self, note, velocity=127, osc=None):
        self.note = note
        self.velocity = velocity
        self.osc = osc

    def __eq__(self, __value: object) -> bool:
        if __value is None:
            return False
        if isinstance(__value, int):
            return self.note == __value
        return self.note == __value.note

    def __lt__(self, __value: object) -> bool:
        return self.note < __value.note

    def __gt__(self, __value: object) -> bool:
        return self.note > __value.note

    def __le__(self, __value: object) -> bool:
        return self.note <= __value.note

    def __ge__(self, __value: object) -> bool:
        return self.note >= __value.note

    def __str__(self):
        note, octave = self.number_to_note(self.note)
        return f'{note}{octave}'

    def __repr__(self) -> str:
        return self.__str__()

    def number_to_note(self, number: int) -> tuple:
        octave = number // self.NOTES_IN_OCTAVE
        note = self.NOTES[number % self.NOTES_IN_OCTAVE]
        return note, octave

In [3]:
class Oscillator:
    octave_dict = {-1: 0, 0: 42, 1: 84, 2: 127}
    def __init__(self, monologue_controller, tuning_channel, octave_channel, volume_channel):
        self.monologue_controller = monologue_controller

        self.tuning_channel = tuning_channel
        self.octave_channel = octave_channel
        self.volume_channel = volume_channel

        self.tuning_semitones = 0
        self.volume = 127
        self.note_playing = None

    def start_note(self, note):
        self.note_playing = self.monologue_controller.start_note_on(self, note)

    def stop_note(self):
        self.note_playing = self.monologue_controller.stop_note_on(self, self.note_playing)

    def set_tuning(self, semitones):
        self.tuning_semitones = semitones

        semitone_diff = semitones
        octave_diff = 0

        if abs(semitone_diff) > 12:
            octave_diff = semitone_diff // 12
            semitone_diff = semitone_diff % 12

        if semitone_diff == 0:
            cv = 64
        elif semitone_diff > 0:
            if semitone_diff == 1:
                cv = 83
            else:
                cv = 83 + 4 * (semitone_diff - 1)
        else:
            if semitone_diff == -1:
                cv = 44
            else:
                cv = 44 + 4 * (semitone_diff + 1)

        self.monologue_controller.control_change(self.tuning_channel, cv)
        self.monologue_controller.control_change(self.octave_channel, self.octave_dict.get(octave_diff, 42))

    def set_volume(self, volume):
        self.volume = volume
        self.monologue_controller.control_change(self.volume_channel, volume)


In [4]:
class MonologueController:
    def __init__(self, out_port):
        self.notes_pressed = []
        self.oscillators = [Oscillator(self, 34, 48, 39), Oscillator(self, 35, 49, 40)]
        self.notes_playing = [None, None] # [osc1, osc2]

        self.out_port = out_port

    def control_change(self, control, value):
        self.out_port.send(mido.Message('control_change', control=control, value=value))

    def start_note_on(self, osc: Oscillator, note: Note):
        if osc is self.oscillators[0]:
            self.start_note_on_osc1(note)
        elif osc is self.oscillators[1]:
            self.start_note_on_osc2(note)
        return note

    def stop_note_on(self, osc: Oscillator, note: Note):
        if osc is self.oscillators[0]:
            self.stop_note_on_osc1()
        elif osc is self.oscillators[1]:
            self.stop_note_on_osc2()
        return note

    def start_note_on_osc1(self, note: Note):
        if note == self.notes_playing[0]:
            return
        if self.notes_playing[1] is None:
            self.oscillators[0].set_tuning(0)
            self.oscillators[0].set_volume(note.velocity)
            self.note_on(note)
        else:
            osc2_note = self.notes_playing[1]
            self.stop_note_on_osc2()
            self.start_note_on_osc1(note) # FIXME this is not the best way to do it, we can detune osc1
            self.start_note_on_osc2(osc2_note)
        self.notes_playing[0] = note
        print('start_note_on_osc1', note, self.notes_playing, self.notes_pressed)

    def stop_note_on_osc1(self):
        print('stop_note_on_osc1')
        self.oscillators[0].set_volume(0)
        if self.notes_playing[1] is None:
            self.all_note_off()
        self.notes_playing[0] = None

    def start_note_on_osc2(self, note: Note):
        if note == self.notes_playing[1]:
            return
        if self.notes_playing[0] is None:
            self.oscillators[1].set_tuning(0)
            self.oscillators[1].set_volume(note.velocity)
            self.oscillators[0].set_volume(0)
            self.note_on(note)
        else:
            self.oscillators[1].set_tuning(note.note - self.notes_playing[0].note)
            self.oscillators[1].set_volume(note.velocity)
        self.notes_playing[1] = note
        print('start_note_on_osc2', note, self.notes_playing, self.notes_pressed)

    def stop_note_on_osc2(self):
        print('stop_note_on_osc2')
        self.oscillators[1].set_volume(0)
        if self.notes_playing[0] is None:
            self.all_note_off()
        self.notes_playing[1] = None

    def note_on(self, note: Note):
        self.out_port.send(mido.Message('note_on', note=note.note))

    def all_note_off(self):
        print('all_note_off')
        self.control_change(123, 0)
        self.oscillators[0].set_volume(0)
        self.oscillators[1].set_volume(0)
        self.notes_playing = [None, None]

In [13]:
class PolyphonyController(ABC):
    def __init__(self, monologue: MonologueController, in_port):
        self.monologue = monologue
        self.in_port = in_port

        self.notes_pressed = []

    def parse_midi(self, msg):
        if msg is None:
            return
        if msg.type == 'clock':
            return
        if msg.type == 'note_on' or msg.type == 'note_off':
            note = Note(msg.note, msg.velocity)
            if msg.type == 'note_on':
                if note not in self.notes_pressed:
                    self.notes_pressed.append(note)
                self.add_note_pressed(note)
            elif msg.type == 'note_off':
                if note in self.notes_pressed:
                    self.notes_pressed.remove(note)
                self.remove_note_pressed(note)

    @abstractmethod
    def add_note_pressed(self, note: Note):
        pass

    @abstractmethod
    def remove_note_pressed(self, note: Note):
        pass

    def run(self):
        for msg in self.in_port:
            self.parse_midi(msg)

In [14]:
class BassPriorityPolyphonyController(PolyphonyController):
    def __init__(self, monologue: MonologueController, in_port):
        super().__init__(monologue, in_port)

    def add_note_pressed(self, note: Note):
        if len(self.notes_pressed) == 1:
            self.monologue.start_note_on_osc2(note)
        else:
            low_note = min(self.notes_pressed)
            other_notes = [note for note in self.notes_pressed if note != low_note]
            high_note = other_notes[-1]
            self.monologue.start_note_on_osc1(low_note)
            self.monologue.start_note_on_osc2(high_note)

    def remove_note_pressed(self, note: Note):
        if len(self.notes_pressed) == 0:
            self.monologue.all_note_off()
        elif note == self.monologue.notes_playing[0]:
            self.monologue.stop_note_on_osc1()
            if len(self.notes_pressed) > 1:
                self.monologue.start_note_on_osc1(min(self.notes_pressed))
        elif note == self.monologue.notes_playing[1]:
            self.monologue.stop_note_on_osc2()
            if len(self.notes_pressed) > 1:
                self.monologue.start_note_on_osc2(self.notes_pressed[-1])

In [19]:
class MonophonicController(PolyphonyController):
    def __init__(self, monologue: MonologueController, in_port, oscillator: Oscillator):
        super().__init__(monologue, in_port)
        self.oscillator = oscillator

    def add_note_pressed(self, note: Note):
        self.oscillator.start_note(note)

    def remove_note_pressed(self, note: Note):
        if not self.notes_pressed:
            self.oscillator.stop_note()
        else:
            self.oscillator.start_note(self.notes_pressed[-1])

In [8]:
out_port = mido.open_output('monologue SOUND')
in_port = mido.open_input('monologue KBD/KNOB')


In [10]:
microkey = mido.open_input('microKEY-25 KEYBOARD')

In [22]:
monologue_controller = MonologueController(out_port)
# polyphony_controller = BassPriorityPolyphonyController(monologue_controller, in_port)
# polyphony_controller.run()
microkey_controller = MonophonicController(monologue_controller, microkey, monologue_controller.oscillators[0])
monologue_keyboard_controller = MonophonicController(monologue_controller, in_port, monologue_controller.oscillators[1])

while True:
    microkey_controller.parse_midi(microkey.poll())
    monologue_keyboard_controller.parse_midi(in_port.poll())

start_note_on_osc1 E4 [E4, None] []


TypeError: MonologueController.stop_note_on_osc1() takes 1 positional argument but 2 were given